In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Ceará - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [5]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_CE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Ceará - Produção de Cimento (t)', axis=1)
data

,Ano,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - IDH Longevidade,Ceará - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Ceará - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH,Ceará - Consumo de Cimento (t)
0,2008,7.430439e+07,4.781538e+06,7.686809,6.447530e+07,0.788078,0.686465,0.659869,5.483335e+08,2.546644e+09,...,8.024047,19.530149,1008.923849,1487.946810,27.057592,21.627565,73.260756,2.648291,0.705344,1328.591000
1,2009,7.487461e+07,4.827502e+06,7.698375,6.456035e+07,0.790273,0.691475,0.660519,5.302764e+08,2.679945e+09,...,8.052145,19.393252,1008.908015,1398.714107,27.170444,21.546547,72.588532,2.613315,0.708210,1437.485000
2,2010,7.682885e+07,4.851802e+06,7.744071,6.510367e+07,0.791872,0.696107,0.660850,5.046425e+08,2.891930e+09,...,8.017491,19.495931,1008.929298,1352.609172,27.206178,21.476514,72.156276,2.630976,0.710635,1794.240000
3,2011,7.858864e+07,4.848876e+06,7.773847,6.550376e+07,0.792885,0.700356,0.660869,4.854622e+08,3.091988e+09,...,8.006912,18.136683,1009.081267,1353.481052,27.247003,21.498658,72.034757,2.712631,0.712630,1717.740000
4,2012,8.015440e+07,4.829775e+06,7.791383,6.580570e+07,0.793321,0.704333,0.660403,4.867661e+08,3.189506e+09,...,9.818854,17.755013,1009.208938,1324.118308,27.303325,21.513635,71.799537,2.739656,0.714175,1903.741000
5,2013,8.160149e+07,4.793349e+06,7.811016,6.610515e+07,0.793098,0.707444,0.659936,5.225848e+08,3.093872e+09,...,10.221864,18.955957,1009.427497,1294.208198,27.279320,21.416356,71.445993,2.723080,0.715273,2065.389000
6,2014,8.275393e+07,4.756888e+06,7.821411,6.627704e+07,0.792439,0.710814,0.659347,5.676413e+08,2.779340e+09,...,10.652422,18.599712,1009.682618,1313.782408,27.429527,21.316386,70.443006,2.652161,0.716129,1964.413143
7,2015,8.340186e+07,4.705636e+06,7.807860,6.621518e+07,0.791476,0.713466,0.657960,5.179392e+08,2.561731e+09,...,11.020421,18.179780,1009.682825,1316.471785,27.559921,21.342508,70.089489,2.538890,0.716639,2037.703714
8,2016,8.386853e+07,4.627836e+06,7.802392,6.619932e+07,0.802758,0.730642,0.662422,4.524144e+08,2.516859e+09,...,11.073199,19.751583,1009.575611,1278.441481,27.460532,21.334656,70.525466,2.458740,0.729111,2146.547143
9,2017,8.401644e+07,4.573568e+06,7.799909,6.618240e+07,0.801917,0.732700,0.662461,4.213689e+08,2.497710e+09,...,10.712887,21.401573,1009.628585,1268.231186,27.368964,21.410344,71.183860,2.400073,0.729757,2035.158000


In [6]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - IDH Longevidade,Ceará - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Ceará - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH
0,7.430439e+07,4.781538e+06,7.686809,6.447530e+07,0.788078,0.686465,0.659869,5.483335e+08,2.546644e+09,1.874883e+10,...,11.760983,8.024047,19.530149,1008.923849,1487.946810,27.057592,21.627565,73.260756,2.648291,0.705344
1,7.487461e+07,4.827502e+06,7.698375,6.456035e+07,0.790273,0.691475,0.660519,5.302764e+08,2.679945e+09,2.055015e+10,...,9.472585,8.052145,19.393252,1008.908015,1398.714107,27.170444,21.546547,72.588532,2.613315,0.708210
2,7.682885e+07,4.851802e+06,7.744071,6.510367e+07,0.791872,0.696107,0.660850,5.046425e+08,2.891930e+09,2.342247e+10,...,9.332485,8.017491,19.495931,1008.929298,1352.609172,27.206178,21.476514,72.156276,2.630976,0.710635
3,7.858864e+07,4.848876e+06,7.773847,6.550376e+07,0.792885,0.700356,0.660869,4.854622e+08,3.091988e+09,2.604265e+10,...,10.985689,8.006912,18.136683,1009.081267,1353.481052,27.247003,21.498658,72.034757,2.712631,0.712630
4,8.015440e+07,4.829775e+06,7.791383,6.580570e+07,0.793321,0.704333,0.660403,4.867661e+08,3.189506e+09,2.708753e+10,...,8.159014,9.818854,17.755013,1009.208938,1324.118308,27.303325,21.513635,71.799537,2.739656,0.714175
5,8.160149e+07,4.793349e+06,7.811016,6.610515e+07,0.793098,0.707444,0.659936,5.225848e+08,3.093872e+09,2.523394e+10,...,7.913561,10.221864,18.955957,1009.427497,1294.208198,27.279320,21.416356,71.445993,2.723080,0.715273
6,8.275393e+07,4.756888e+06,7.821411,6.627704e+07,0.792439,0.710814,0.659347,5.676413e+08,2.779340e+09,1.954361e+10,...,10.350442,10.652422,18.599712,1009.682618,1313.782408,27.429527,21.316386,70.443006,2.652161,0.716129
7,8.340186e+07,4.705636e+06,7.807860,6.621518e+07,0.791476,0.713466,0.657960,5.179392e+08,2.561731e+09,1.327596e+10,...,12.454398,11.020421,18.179780,1009.682825,1316.471785,27.559921,21.342508,70.089489,2.538890,0.716639
8,8.386853e+07,4.627836e+06,7.802392,6.619932e+07,0.802758,0.730642,0.662422,4.524144e+08,2.516859e+09,1.011370e+10,...,13.111908,11.073199,19.751583,1009.575611,1278.441481,27.460532,21.334656,70.525466,2.458740,0.729111
9,8.401644e+07,4.573568e+06,7.799909,6.618240e+07,0.801917,0.732700,0.662461,4.213689e+08,2.497710e+09,8.456309e+09,...,9.479812,10.712887,21.401573,1009.628585,1268.231186,27.368964,21.410344,71.183860,2.400073,0.729757


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1437.485000
1     1794.240000
2     1717.740000
3     1903.741000
4     2065.389000
5     1964.413143
6     2037.703714
7     2146.547143
8     2035.158000
9     1758.900000
10    1701.230000
11    1864.364000
12    2127.104000
13    1966.646000
14            NaN
Name: Ceará - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - IDH Longevidade,Ceará - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Ceará - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH
0,-1.825466,0.487152,-2.082162,-1.966996,-1.432506,-1.530955,-0.598881,1.204752,-0.990234,0.166091,...,0.984445,-1.370027,-0.037551,-1.397395,2.361873,-1.968241,1.645394,1.854362,0.575704,-1.479238
1,-1.660185,0.916292,-1.805814,-1.829234,-0.965438,-1.228499,-0.056326,0.820657,-0.546688,0.428911,...,-0.051710,-1.347480,-0.135767,-1.449574,1.110791,-1.102522,0.815692,1.097108,0.316306,-1.145473
2,-1.093737,1.143174,-0.713987,-0.949131,-0.625247,-0.948873,0.220138,0.275392,0.158672,0.847995,...,-0.115145,-1.375288,-0.062101,-1.379437,0.464379,-0.828394,0.098490,0.610176,0.447285,-0.863001
3,-0.583651,1.115853,-0.002554,-0.301050,-0.409551,-0.692386,0.236503,-0.132596,0.824346,1.230291,...,0.633403,-1.383777,-1.037284,-0.878631,0.476603,-0.515214,0.325264,0.473287,1.052873,-0.630588
4,-0.129809,0.937515,0.416452,0.188047,-0.316832,-0.452299,-0.153228,-0.104860,1.148829,1.382743,...,-0.646476,0.070207,-1.311110,-0.457901,0.064924,-0.083159,0.478640,0.208314,1.253302,-0.450685
5,0.289639,0.597428,0.885541,0.673111,-0.364332,-0.264486,-0.542565,0.657046,0.830615,1.112297,...,-0.757614,0.393601,-0.449501,0.262347,-0.354429,-0.267309,-0.517588,-0.189950,1.130368,-0.322806
6,0.623680,0.257006,1.133921,0.951549,-0.504446,-0.061023,-1.033902,1.615453,-0.215960,0.282052,...,0.345772,0.739100,-0.705087,1.103081,-0.079990,0.884973,-1.541380,-1.319806,0.604407,-0.223096
7,0.811486,-0.221506,0.810123,0.851339,-0.709430,0.099028,-2.192189,0.558230,-0.940033,-0.632425,...,1.298413,1.034399,-1.006364,1.103764,-0.042283,1.885254,-1.273864,-1.718039,-0.235662,-0.163608
8,0.946754,-0.947874,0.679480,0.825655,1.691026,1.135918,1.532636,-0.835563,-1.089341,-1.093812,...,1.596124,1.076751,0.121315,0.750445,-0.575485,1.122822,-1.354272,-1.226915,-0.830082,1.288969
9,0.989625,-1.454550,0.620159,0.798254,1.512250,1.260177,1.564684,-1.495939,-1.153056,-1.335633,...,-0.048438,0.787620,1.305088,0.925018,-0.718638,0.420374,-0.579156,-0.485241,-1.265186,1.364260


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     1437.485000
1     1794.240000
2     1717.740000
3     1903.741000
4     2065.389000
5     1964.413143
6     2037.703714
7     2146.547143
8     2035.158000
9     1758.900000
10    1701.230000
11    1864.364000
Name: Ceará - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - IDH Longevidade,Ceará - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Ceará - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH
12,0.373891,-1.122598,-0.818152,-0.27511,0.400623,1.168563,-0.47095,1.108405,2.635823,-0.189959,...,-2.260447,0.613586,1.025085,-0.211439,-1.463016,0.503155,1.032998,0.281873,-1.020017,1.013884


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    2127.104
Name: Ceará - Consumo de Cimento (t), dtype: float64

In [17]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        tf.keras.utils.set_random_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.0025)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [18]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [19]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2389495888, 1888910726, 356193572, 3389547659, 679051041, 4210698893, 3536392928, 1840746421, 1324446079, 1270076785, 2259600656, 2295791242, 342866206, 3034799119, 3014281400, 969765293, 1864259176, 2512026135, 753044568, 1464267661, 3092031853, 1175231389, 2389774742, 3896942787, 1560038230, 3136313714, 759594637, 1083451529, 1034404320, 3406285574, 655344777, 3646932203, 2939057319, 3502836516, 331812063, 1178487735, 1961498979, 1661818253, 1481114114, 2284892101, 2033345025, 1531334298, 2469764754, 2585245571, 2618604387, 1554977683, 2402942160, 2480470142, 1017970688, 3525300496, 3816175628, 3641295439, 936064310, 3230887122, 3974119106, 2391463024, 2413765535, 2838867318, 2719370110, 3612814530, 1087144043, 3317107332, 3368430192, 751499245, 1736724096, 984847195, 221747525, 2124583522, 172599174, 410493153, 742417192, 695682195, 1605910644, 3067728705, 1694714150, 564114713, 1134669999, 1325160646, 881015554, 83830026, 810888752, 3587413407, 2932226902, 311247001, 2944146178, 2

2023-09-14 18:44:02.426145: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-14 18:44:02.457771: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-14 18:44:02.491700: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 0.009135738015174866
winner_seed: 1888910726


Step: 2 ___________________________________________
val_loss: 0.005746498703956604
winner_seed: 356193572


Step: 3 ___________________________________________
val_loss: 364.5153503417969


Step: 4 ___________________________________________
val_loss: 0.00048279762268066406
winner_seed: 679051041


Step: 5 ___________________________________________
val_loss: 0.023023024201393127


Step: 6 ___________________________________________
val_loss: 1.9311904907226562e-05
winner_seed: 3536392928


Step: 7 ___________________________________________
val_loss: 0.008093848824501038


Step: 8 ___________________________________________
val_loss: 858.9578857421875


Step: 9 ___________________________________________
val_loss: 0.00018359720706939697


Step: 10 ___________________________________________
val_loss: 4.812967300415039


Step: 11 ___________________________________________
val_loss: 13.469223022460938


Step: 12 _________________

val_loss: 447.5042724609375


Step: 98 ___________________________________________
val_loss: 0.0006323456764221191


Step: 99 ___________________________________________
val_loss: 3.8774187564849854


final_seed: 984847195


In [20]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 487ms/step - loss: 3528606.0000 - val_loss: 3461602.0000
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 3465320.0000 - val_loss: 2836192.0000
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 3569274.2500 - val_loss: 3524123.0000
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 3429176.0000 - val_loss: 3515978.5000
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 3361165.2500 - val_loss: 3627545.7500
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 3311718.0000 - val_loss: 3450865.5000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 3290235.2500 - val_loss: 3265957.5000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 3238336.7500 - val_loss: 3233292.5000
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 3208826.7500 - val_loss: 3567

1/1 [==============================] - 0s 14ms/step - loss: 1329146.5000 - val_loss: 2481401.7500
Epoch 74/10000
1/1 [==============================] - 0s 14ms/step - loss: 1358136.6250 - val_loss: 2346664.5000
Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 1410619.3750 - val_loss: 2108215.7500
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 1434885.1250 - val_loss: 2253175.0000
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 1322367.2500 - val_loss: 2257882.5000
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 1389486.6250 - val_loss: 2163197.5000
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 1367870.6250 - val_loss: 1887266.5000
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 1322952.2500 - val_loss: 2197425.5000
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 1280576.5000 - val_loss: 2163161.250

1/1 [==============================] - 0s 14ms/step - loss: 729342.2500 - val_loss: 840203.1875
Epoch 146/10000
1/1 [==============================] - 0s 14ms/step - loss: 735519.5000 - val_loss: 822394.1250
Epoch 147/10000
1/1 [==============================] - 0s 14ms/step - loss: 830597.1875 - val_loss: 658729.5625
Epoch 148/10000
1/1 [==============================] - 0s 14ms/step - loss: 781012.0000 - val_loss: 697994.7500
Epoch 149/10000
1/1 [==============================] - 0s 13ms/step - loss: 756601.1875 - val_loss: 729673.3750
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 675025.0625 - val_loss: 778204.0625
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 720942.5000 - val_loss: 774084.7500
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 697090.5625 - val_loss: 771179.0625
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 827731.4375 - val_loss: 1007809.2500
Epoch 1

1/1 [==============================] - 0s 14ms/step - loss: 399191.0000 - val_loss: 643564.1250
Epoch 219/10000
1/1 [==============================] - 0s 14ms/step - loss: 413681.5938 - val_loss: 754561.3750
Epoch 220/10000
1/1 [==============================] - 0s 13ms/step - loss: 440281.0000 - val_loss: 629839.0625
Epoch 221/10000
1/1 [==============================] - 0s 13ms/step - loss: 431476.8438 - val_loss: 659217.3125
Epoch 222/10000
1/1 [==============================] - 0s 13ms/step - loss: 444340.9688 - val_loss: 639910.1875
Epoch 223/10000
1/1 [==============================] - 0s 13ms/step - loss: 461528.5000 - val_loss: 679871.1250
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 418638.3438 - val_loss: 667761.5000
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 415481.5312 - val_loss: 658016.0625
Epoch 226/10000
1/1 [==============================] - 0s 13ms/step - loss: 412319.1562 - val_loss: 648927.8125
Epoch 22

1/1 [==============================] - 0s 14ms/step - loss: 249776.9844 - val_loss: 283033.8750
Epoch 292/10000
1/1 [==============================] - 0s 13ms/step - loss: 247797.5469 - val_loss: 285443.0312
Epoch 293/10000
1/1 [==============================] - 0s 13ms/step - loss: 250367.1094 - val_loss: 280278.8750
Epoch 294/10000
1/1 [==============================] - 0s 13ms/step - loss: 247196.0000 - val_loss: 274755.3750
Epoch 295/10000
1/1 [==============================] - 0s 13ms/step - loss: 241548.6250 - val_loss: 277706.5000
Epoch 296/10000
1/1 [==============================] - 0s 13ms/step - loss: 260664.1094 - val_loss: 254934.0156
Epoch 297/10000
1/1 [==============================] - 0s 13ms/step - loss: 240440.0781 - val_loss: 267040.4062
Epoch 298/10000
1/1 [==============================] - 0s 13ms/step - loss: 236767.0781 - val_loss: 272775.2812
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 236615.4844 - val_loss: 271609.6875
Epoch 30

...

1/1 [==============================] - 0s 14ms/step - loss: 22723.2656 - val_loss: 9949.0889
Epoch 1400/10000
1/1 [==============================] - 0s 14ms/step - loss: 22952.5156 - val_loss: 10375.9541
Epoch 1401/10000
1/1 [==============================] - 0s 14ms/step - loss: 22688.7754 - val_loss: 10527.4561
Epoch 1402/10000
1/1 [==============================] - 0s 14ms/step - loss: 22945.2324 - val_loss: 10639.7012
Epoch 1403/10000
1/1 [==============================] - 0s 14ms/step - loss: 22696.1309 - val_loss: 10473.0439
Epoch 1404/10000
1/1 [==============================] - 0s 14ms/step - loss: 22686.7051 - val_loss: 10564.5127
Epoch 1405/10000
1/1 [==============================] - 0s 14ms/step - loss: 22660.1348 - val_loss: 10383.1670
Epoch 1406/10000
1/1 [==============================] - 0s 14ms/step - loss: 22703.6562 - val_loss: 10330.0479
Epoch 1407/10000
1/1 [==============================] - 0s 14ms/step - loss: 22722.6934 - val_loss: 10480.8408
Epoch 1408/10000
1/

Epoch 1473/10000
1/1 [==============================] - 0s 14ms/step - loss: 22685.5098 - val_loss: 10054.8604
Epoch 1474/10000
1/1 [==============================] - 0s 14ms/step - loss: 22666.3027 - val_loss: 10044.1895
Epoch 1475/10000
1/1 [==============================] - 0s 14ms/step - loss: 22531.9590 - val_loss: 9603.1152
Epoch 1476/10000
1/1 [==============================] - 0s 14ms/step - loss: 22665.8203 - val_loss: 9818.3857
Epoch 1477/10000
1/1 [==============================] - 0s 14ms/step - loss: 22662.4062 - val_loss: 9914.6846
Epoch 1478/10000
1/1 [==============================] - 0s 14ms/step - loss: 22674.3184 - val_loss: 9992.7256
Epoch 1479/10000
1/1 [==============================] - 0s 14ms/step - loss: 22660.9941 - val_loss: 9987.4795
Epoch 1480/10000
1/1 [==============================] - 0s 14ms/step - loss: 22680.4609 - val_loss: 10138.3418
Epoch 1481/10000
1/1 [==============================] - 0s 14ms/step - loss: 22681.9355 - val_loss: 10055.7412
Epoch 

1/1 [==============================] - 0s 14ms/step - loss: 23849.6191 - val_loss: 8918.9023
Epoch 1548/10000
1/1 [==============================] - 0s 14ms/step - loss: 23829.3887 - val_loss: 8988.8086
Epoch 1549/10000
1/1 [==============================] - 0s 13ms/step - loss: 23776.6562 - val_loss: 9017.7646
Epoch 1550/10000
1/1 [==============================] - 0s 13ms/step - loss: 23763.0762 - val_loss: 9048.4404
Epoch 1551/10000
1/1 [==============================] - 0s 13ms/step - loss: 23742.8359 - val_loss: 9064.5645
Epoch 1552/10000
1/1 [==============================] - 0s 13ms/step - loss: 25239.7012 - val_loss: 7029.7334
Epoch 1553/10000
1/1 [==============================] - 0s 14ms/step - loss: 24225.7402 - val_loss: 7858.9351
Epoch 1554/10000
1/1 [==============================] - 0s 14ms/step - loss: 26846.2910 - val_loss: 7371.6646
Epoch 1555/10000
1/1 [==============================] - 0s 13ms/step - loss: 23807.3223 - val_loss: 8247.4863
Epoch 1556/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 23102.1855 - val_loss: 8616.9248
Epoch 1622/10000
1/1 [==============================] - 0s 13ms/step - loss: 23034.0801 - val_loss: 9014.3340
Epoch 1623/10000
1/1 [==============================] - 0s 14ms/step - loss: 23021.4941 - val_loss: 9224.0410
Epoch 1624/10000
1/1 [==============================] - 0s 14ms/step - loss: 23024.1504 - val_loss: 9323.8916
Epoch 1625/10000
1/1 [==============================] - 0s 14ms/step - loss: 23497.8496 - val_loss: 9031.9590
Epoch 1626/10000
1/1 [==============================] - 0s 14ms/step - loss: 22998.9629 - val_loss: 9191.6182
Epoch 1627/10000
1/1 [==============================] - 0s 14ms/step - loss: 23007.7812 - val_loss: 9299.9795
Epoch 1628/10000
1/1 [==============================] - 0s 13ms/step - loss: 23001.4277 - val_loss: 9344.4834
Epoch 1629/10000
1/1 [==============================] - 0s 14ms/step - loss: 22979.7402 - val_loss: 9316.6797
Epoch 1630/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 23172.5469 - val_loss: 9157.0322
Epoch 1696/10000
1/1 [==============================] - 0s 15ms/step - loss: 23943.4453 - val_loss: 8700.0703
Epoch 1697/10000
1/1 [==============================] - 0s 15ms/step - loss: 23161.2637 - val_loss: 8933.8730
Epoch 1698/10000
1/1 [==============================] - 0s 26ms/step - loss: 23051.1270 - val_loss: 9103.5400
Epoch 1698: early stopping


In [24]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 22ms/step
[2127.104](test_target) - [[1864.3658]](prediction) = 262.7381552734373


In [25]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [26]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - IDH Longevidade,Ceará - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Ceará - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH
0,-0.954262,-1.329710,-0.928006,-0.854355,-1.283635,-1.240439,-1.334114,1.148205,-1.122043,-1.121418,...,1.412346,-0.477724,0.980535,0.383452,1.332476,-1.376174,1.253331,1.303231,1.220695,-1.257275
1,-0.426771,0.247838,-0.460175,-0.548813,0.127803,0.032016,0.260725,0.140893,-0.184474,-0.185492,...,-0.643250,1.391613,-1.372828,-1.370591,-0.255896,0.405937,-0.059328,-0.176019,-1.228755,0.067883
2,1.381033,1.081872,1.388180,1.403168,1.155832,1.208423,1.073389,-1.289098,1.306518,1.306910,...,-0.769096,-0.913889,0.392294,0.987139,-1.076580,0.970237,-1.194003,-1.127212,0.008060,1.189392


train_target:


,Ceará - Consumo de Cimento (t)
0,1437.485
1,1794.240
2,1717.740


test_input:


...

,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - IDH Longevidade,Ceará - IDH Educacao,Bahia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Ceará - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH
12,0.373891,-1.122598,-0.818152,-0.27511,0.400623,1.168563,-0.47095,1.108405,2.635823,-0.189959,...,-2.260447,0.613586,1.025085,-0.211439,-1.463016,0.503155,1.032998,0.281873,-1.020017,1.013884


test_target:


,Ceará - Consumo de Cimento (t)
12,2127.104


1/1 [==============================] - 0s 35ms/step
Error: 262.7381552734373




[1717.7501220703125,
 1810.7283935546875,
 1900.5137939453125,
 1964.381103515625,
 2037.8397216796875,
 2069.409423828125,
 2034.136474609375,
 1758.5758056640625,
 1701.21484375,
 1864.3658447265625]

In [27]:
display(targets)
display(predictions)

[1903.741,
 2065.389,
 1964.4131428571427,
 2037.703714285714,
 2146.5471428571427,
 2035.158,
 1758.9,
 1701.23,
 1864.364,
 2127.104]

[1717.7501220703125,
 1810.7283935546875,
 1900.5137939453125,
 1964.381103515625,
 2037.8397216796875,
 2069.409423828125,
 2034.136474609375,
 1758.5758056640625,
 1701.21484375,
 1864.3658447265625]

In [28]:
mae = mean_absolute_error(predictions, targets)
mae

147.93018808593746

In [29]:
porcentage = mae/np.mean(targets)
porcentage

0.07545706893855633